## Set up

In [3]:
import pandas as pd

In [4]:
import time

In [5]:
from langchain.llms import Ollama
from langchain_community.chat_models import ChatOllama
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

In [6]:
from langchain.vectorstores import Chroma
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [7]:
from langchain.embeddings import HuggingFaceEmbeddings

In [8]:
import importlib
import myloadlib
import myutils
import Vector_database_functions as db

In [9]:
import ChatTestDB as log
import Harry_Potter_questions as HP

## Embeddings

In [10]:
model_name = "sentence-transformers/all-mpnet-base-v2"
# model_name = "sentence-transformers/all-MiniLM-l6-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}

In [11]:
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

## Get data

In [12]:
#get databasen
vectordb = db.get_db( "Harry_Potter", "data/Chroma/", embeddings)

In [13]:
len(vectordb)

958318

# LLM

In [ ]:
# We have tried with diffirent models

In [ ]:
#llm = Ollama(model="gemma", callback_manager = CallbackManager([StreamingStdOutCallbackHandler()]))

In [ ]:
#llm = Ollama(model="zephyr", callback_manager = CallbackManager([StreamingStdOutCallbackHandler()]))

In [39]:
#llm = Ollama(model="llama3", callback_manager = CallbackManager([StreamingStdOutCallbackHandler()]))

In [44]:
#llm = Ollama(model="mistral", callback_manager = CallbackManager([StreamingStdOutCallbackHandler()]))

In [47]:
#llm = ChatOllama(model="mistral")

In [12]:
llm = ChatOllama(model="llama3")

In [13]:
#llm = ChatOllama(model="tomasonjo/llama3-text2cypher-demo")

In [14]:
# Build prompt
template = """Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know, don't try to make up an answer. 
Use five sentences maximum. Keep the answer as concise as possible. 
Always say "thanks for asking!" at the end of the answer. 

{context}

Question: {question}

Helpful Answer:
"""

In [23]:
question = "what is harry potters wand?"

In [ ]:
question = "which hogwarts house does Harry belong to?"

In [ ]:
question = "who is Merope Gaunt?"

In [16]:
question = "who is Merope Gaunts son?"

In [48]:
prompt = PromptTemplate.from_template(template)
chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt})

In [49]:
start = time.time()
result = chain.invoke({"query": question})
end = time.time()
print(result["result"])
print((end - start)/60)

 Merope Gaunt is the sister of Morfin Gaunt and Aristina Gaunt in the Harry Potter series. She married a Muggle, Tom Riddle senior, giving birth to their son, Tom Marvolo Riddle, later known as Lord Voldemort. (Thanks for asking!)
13.340355932712555


In [19]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [42]:
rag_chain = (
    {"context": vectordb.as_retriever(), "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [43]:
start = time.time()
result = rag_chain.invoke(question)
end = time.time()
print(result)
print((end - start)/60)

Merope Gaunt's son is Marvolo Gaunt. Thanks for asking!Merope Gaunt's son is Marvolo Gaunt. Thanks for asking!
11.971413230895996


# Test of awnsers

In [15]:
prompt = PromptTemplate.from_template(template)
chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt})

In [37]:
start = time.time()
result = chain.invoke({"query": HP.question1})
end = time.time()
times = (end - start)/60
log.LogChat(2, "vector", HP.question1, result['result'], times)
print(result["result"])
print(times)

According to the LEGO Harry Potter series, Harry Potter's wand is described as being made of holly wood with a phoenix feather core. Thanks for asking!
4.723734839757284


In [38]:
start = time.time()
result = chain.invoke({"query": HP.question2})
end = time.time()
times = (end - start)/60
log.LogChat(2, "vector", HP.question2, result['result'], times)
print(result["result"])
print(times)

According to the context, there is no mention of Harry Potter's Hogwarts house being discussed. However, we do know that Jacob's sibling chooses their own Hogwarts house, and Harry and Ginevra Potter's house is mentioned as having a connection to Hogwarts.

I don't know the answer to this specific question.

Thanks for asking!
3.929632031917572


In [39]:
start = time.time()
result = chain.invoke({"query": HP.question3})
end = time.time()
times = (end - start)/60
log.LogChat(2, "vector", HP.question3, result['result'], times)
print(result["result"])
print(times)

Merope Gaunt was Bellatrix Lestrange's mother. She was a witch who had an affair with Marvolo Gaunt and had Bellatrix as a result. Thanks for asking!
0.8534922003746033


In [40]:
start = time.time()
result = chain.invoke({"query": HP.question4})
end = time.time()
times = (end - start)/60
log.LogChat(2, "vector", HP.question4, result['result'], times)
print(result["result"])
print(times)

I don't know who Merope Gaunt's son is. The context provided does not mention any information about her children or their names. Thanks for asking!
0.8249166250228882


In [41]:
start = time.time()
result = chain.invoke({"query": HP.question5})
end = time.time()
times = (end - start)/60
log.LogChat(2, "vector", HP.question5, result['result'], times)
print(result["result"])
print(times)

Fred and Griphook, two of Harry's friends, created the Marauder's Map.

Thanks for asking!
0.6314687887827556


In [42]:
start = time.time()
result = chain.invoke({"query": HP.question6})
end = time.time()
times = (end - start)/60
log.LogChat(2, "vector", HP.question6, result['result'], times)
print(result["result"])
print(times)

The Half-Blood Prince refers to Severus Snape, a complex and intriguing character in the Harry Potter series. He is known for being a potions master at Hogwarts School of Witchcraft and Wizardry. Snape's mother was a witch, while his father was a Muggle (non-magical) person, making him a half-blood. His true loyalties are gradually revealed throughout the series. Thanks for asking!
1.670889186859131


In [43]:
start = time.time()
result = chain.invoke({"query": HP.question7})
end = time.time()
times = (end - start)/60
log.LogChat(2, "vector", HP.question7, result['result'], times)
print(result["result"])
print(times)

Unfortunately, I don't know the answer to this question. The context only mentions that someone thought Harry Potter might be the Heir of Slytherin, but it doesn't explicitly state who the Heir actually is. Thanks for asking!
1.1389414509137472


In [44]:
start = time.time()
result = chain.invoke({"query": HP.question8})
end = time.time()
times = (end - start)/60
log.LogChat(2, "vector", HP.question8, result['result'], times)
print(result["result"])
print(times)

I don't know the answer. Thanks for asking!
2.8390076637268065


In [45]:
start = time.time()
result = chain.invoke({"query": HP.question9})
end = time.time()
times = (end - start)/60
log.LogChat(2, "vector", HP.question9, result['result'], times)
print(result["result"])
print(times)

Voldemort created a total of 7 Horcruxes. Thanks for asking!
2.5055452227592467


In [46]:
start = time.time()
result = chain.invoke({"query": HP.question10})
end = time.time()
times = (end - start)/60
log.LogChat(2, "vector", HP.question10, result['result'], times)
print(result["result"])
print(times)

There are three Deathly Hallows. Thanks for asking!
0.32028677463531496


In [47]:
start = time.time()
result = chain.invoke({"query": HP.question11})
end = time.time()
times = (end - start)/60
log.LogChat(2, "vector", HP.question11, result['result'], times)
print(result["result"])
print(times)

The entrance to the Chamber of Secrets is in the second floor bathroom. Thanks for asking!
1.3273857951164245


In [48]:
start = time.time()
result = chain.invoke({"query": HP.question12})
end = time.time()
times = (end - start)/60
log.LogChat(2, "vector", HP.question12, result['result'], times)
print(result["result"])
print(times)

The answer to your question is Dobby. Thanks for asking!
0.6381458282470703


In [49]:
start = time.time()
result = chain.invoke({"query": HP.question13})
end = time.time()
times = (end - start)/60
log.LogChat(2, "vector", HP.question13, result['result'], times)
print(result["result"])
print(times)

I don't know the answer to this question. Thanks for asking!
2.229138954480489


In [50]:
start = time.time()
result = chain.invoke({"query": HP.question14})
end = time.time()
times = (end - start)/60
log.LogChat(2, "vector", HP.question14, result['result'], times)
print(result["result"])
print(times)

According to the given context, Sirius Black is Harry Potter's maternal first cousin, once removed. This means that Sirius was a blood relative of Harry's mother, Lily Evans Potter, making him Harry's maternal cousin. Thanks for asking!
1.072611649831136


In [51]:
start = time.time()
result = chain.invoke({"query": HP.question15})
end = time.time()
times = (end - start)/60
log.LogChat(2, "vector", HP.question15, result['result'], times)
print(result["result"])
print(times)

The answer is Fluffy. Thanks for asking!
2.153825386365255


In [52]:
start = time.time()
result = chain.invoke({"query": HP.question16})
end = time.time()
times = (end - start)/60
log.LogChat(2, "vector", HP.question16, result['result'], times)
print(result["result"])
print(times)

According to the Harry Potter Wiki, the hippogriff that Harry saves from execution is named Buckbeak. Thanks for asking!
1.8011456767717997


In [53]:
start = time.time()
result = chain.invoke({"query": HP.question17})
end = time.time()
times = (end - start)/60
log.LogChat(2, "vector", HP.question17, result['result'], times)
print(result["result"])
print(times)

The ghost of Ravenclaw house is Helena Ravenclaw, the daughter of Rowena Ravenclaw. She is often referred to as "The White Lady". Thanks for asking!
1.9064854621887206


In [54]:
start = time.time()
result = chain.invoke({"query": HP.question18})
end = time.time()
times = (end - start)/60
log.LogChat(2, "vector", HP.question18, result['result'], times)
print(result["result"])
print(times)

Scabbers is the name of Ron's pet rat, which turns out to be an unregistered Animagus named Peter Pettigrew. Thanks for asking!
1.457786003748576


In [55]:
start = time.time()
result = chain.invoke({"query": HP.question19})
end = time.time()
times = (end - start)/60
log.LogChat(2, "vector", HP.question19, result['result'], times)
print(result["result"])
print(times)

There are seven Weasley siblings: Bill, Charlie, Fred, George, Ginny, Molly, and Percy. Thanks for asking!
0.6361682693163554


In [56]:
start = time.time()
result = chain.invoke({"query": HP.question20})
end = time.time()
times = (end - start)/60
log.LogChat(2, "vector", HP.question20, result['result'], times)
print(result["result"])
print(times)

Ginny Weasley is the youngest Weasley sibling. She's the only girl in the family and has a special place in her brothers' hearts. Thanks for asking!
0.610495944817861


# Test of awnsers sentence simularity

In [20]:
docs = vectordb.similarity_search(HP.question2)

In [23]:
print(docs[0].page_content)

Behind the scenes[]
Her house is the same as that of Jacob's sibling, which is chosen by the player.
Appearances[]
Harry Potter: Hogwarts Mystery (First mentioned)
Notes and references[]


↑ "World Exclusive Interview with J K Rowling," South West News Service, 8 July 2000  - "Hogwarts just serves Britain and Ireland."

↑ 2.0 2.1 2.2 2.3 Harry Potter: Hogwarts Mystery, Year 7, Chapter 43 (Father Knows Best)

↑ Harry Potter: Hogwarts Mystery, Year 7, Chapter 48 (Archival Evidence)


In [48]:
question = HP.question1
start = time.time()
result = vectordb.similarity_search(question)
end = time.time()
times = (end - start)/60
log.LogChat(5, "Vector_with_sentence_simularity", question, result[0].page_content, times)
print(result[0].page_content)
print(times)

LEGO Harry Potter: Building the Magical World




LEGO Harry Potter: Characters of the Magical World







Characters
 




Harry Potter




Hermione Granger




Ronald Weasley




Ginny Weasley




Albus Dumbledore




Lord Voldemort




Neville Longbottom




Draco Malfoy




Severus Snape




Rubeus Hagrid











Films

 




The Eight Films
 




Harry Potter and the Philosopher's Stone




Harry Potter and the Chamber of Secrets




Harry Potter and the Prisoner of Azkaban
0.0007878859837849935


In [49]:
question = HP.question2
start = time.time()
result = vectordb.similarity_search(question)
end = time.time()
times = (end - start)/60
log.LogChat(5, "Vector_with_sentence_simularity", question, result[0].page_content, times)
print(result[0].page_content)
print(times)

Behind the scenes[]
Her house is the same as that of Jacob's sibling, which is chosen by the player.
Appearances[]
Harry Potter: Hogwarts Mystery (First mentioned)
Notes and references[]


↑ "World Exclusive Interview with J K Rowling," South West News Service, 8 July 2000  - "Hogwarts just serves Britain and Ireland."

↑ 2.0 2.1 2.2 2.3 Harry Potter: Hogwarts Mystery, Year 7, Chapter 43 (Father Knows Best)

↑ Harry Potter: Hogwarts Mystery, Year 7, Chapter 48 (Archival Evidence)
0.0007474700609842936


In [50]:
question = HP.question3
start = time.time()
result = vectordb.similarity_search(question)
end = time.time()
times = (end - start)/60
log.LogChat(5, "Vector_with_sentence_simularity", question, result[0].page_content, times)
print(result[0].page_content)
print(times)

Bellatrix Lestrange, her mother
0.001641817887624105


In [51]:
question = HP.question4
start = time.time()
result = vectordb.similarity_search(question)
end = time.time()
times = (end - start)/60
log.LogChat(5, "Vector_with_sentence_simularity", question, result[0].page_content, times)
print(result[0].page_content)
print(times)

Merope Gaunt[]
0.0016906181971232095


In [52]:
question = HP.question5
start = time.time()
result = vectordb.similarity_search(question)
end = time.time()
times = (end - start)/60
log.LogChat(5, "Vector_with_sentence_simularity", question, result[0].page_content, times)
print(result[0].page_content)
print(times)

created the Marauder's Map.
0.0018303235371907552


In [53]:
question = HP.question6
start = time.time()
result = vectordb.similarity_search(question)
end = time.time()
times = (end - start)/60
log.LogChat(5, "Vector_with_sentence_simularity", question, result[0].page_content, times)
print(result[0].page_content)
print(times)

Half-Blood Prince[]
0.0018881162007649739


In [54]:
question = HP.question7
start = time.time()
result = vectordb.similarity_search(question)
end = time.time()
times = (end - start)/60
log.LogChat(5, "Vector_with_sentence_simularity", question, result[0].page_content, times)
print(result[0].page_content)
print(times)

The Heir of Slytherin[]
0.0023016492525736493


In [55]:
question = HP.question8
start = time.time()
result = vectordb.similarity_search(question)
end = time.time()
times = (end - start)/60
log.LogChat(5, "Vector_with_sentence_simularity", question, result[0].page_content, times)
print(result[0].page_content)
print(times)

· Jonathan Spratt · Kevin Sterndale · Terry Strickland · Marcus Turner · Gabriel Truman · Percy Weasley · Ron Weasley · Ryan Westrup · Mervyn Wynch · Unidentified Gryffindor Prefect (I) · Unidentified Gryffindor Prefect (II) · Unidentified Ravenclaw Prefect (I) · Unidentified Ravenclaw Prefect (II) · Unidentified Slytherin Prefect (I) · Unidentified Slytherin Prefect (II) · Unidentified Slytherin Prefect (III)
0.0020642797152201336


In [56]:
question = HP.question9
start = time.time()
result = vectordb.similarity_search(question)
end = time.time()
times = (end - start)/60
log.LogChat(5, "Vector_with_sentence_simularity", question, result[0].page_content, times)
print(result[0].page_content)
print(times)

Ronald Weasley




Ginny Weasley




Albus Dumbledore




Lord Voldemort




Neville Longbottom




Draco Malfoy




Severus Snape




Rubeus Hagrid











Films

 




The Eight Films
 




Harry Potter and the Philosopher's Stone




Harry Potter and the Chamber of Secrets




Harry Potter and the Prisoner of Azkaban




Harry Potter and the Goblet of Fire




Harry Potter and the Order of the Phoenix




Harry Potter and the Half-Blood Prince
0.0015284419059753418


In [57]:
question = HP.question10
start = time.time()
result = vectordb.similarity_search(question)
end = time.time()
times = (end - start)/60
log.LogChat(5, "Vector_with_sentence_simularity", question, result[0].page_content, times)
print(result[0].page_content)
print(times)

Ronald Weasley




Ginny Weasley




Albus Dumbledore




Lord Voldemort




Neville Longbottom




Draco Malfoy




Severus Snape




Rubeus Hagrid











Films

 




The Eight Films
 




Harry Potter and the Philosopher's Stone




Harry Potter and the Chamber of Secrets




Harry Potter and the Prisoner of Azkaban




Harry Potter and the Goblet of Fire




Harry Potter and the Order of the Phoenix




Harry Potter and the Half-Blood Prince
0.0015008250872294109


In [58]:
question = HP.question11
start = time.time()
result = vectordb.similarity_search(question)
end = time.time()
times = (end - start)/60
log.LogChat(5, "Vector_with_sentence_simularity", question, result[0].page_content, times)
print(result[0].page_content)
print(times)

Openings[]
    The location of the Chamber of Secrets is in the second floor bathroom
0.001596248149871826


In [59]:
question = HP.question12
start = time.time()
result = vectordb.similarity_search(question)
end = time.time()
times = (end - start)/60
log.LogChat(5, "Vector_with_sentence_simularity", question, result[0].page_content, times)
print(result[0].page_content)
print(times)

Winky, his father's previous house-elf
0.0016001820564270019


In [60]:
question = HP.question13
start = time.time()
result = vectordb.similarity_search(question)
end = time.time()
times = (end - start)/60
log.LogChat(5, "Vector_with_sentence_simularity", question, result[0].page_content, times)
print(result[0].page_content)
print(times)

Unidentified goblin at Gringotts (II) | Harry Potter Wiki | Fandom

















































Harry Potter Wiki





 Explore

 




 Main Page




 Discuss




All Pages




Community




Interactive Maps




Recent Blog Posts








Books

 




The Seven Novels
 




Harry Potter and the Philosopher's Stone




Harry Potter and the Chamber of Secrets




Harry Potter and the Prisoner of Azkaban




Harry Potter and the Goblet of Fire
0.0017157316207885743


In [61]:
question = HP.question14
start = time.time()
result = vectordb.similarity_search(question)
end = time.time()
times = (end - start)/60
log.LogChat(5, "Vector_with_sentence_simularity", question, result[0].page_content, times)
print(result[0].page_content)
print(times)

Sirius Black, his maternal first cousin, once removed
0.0015392502148946126


In [62]:
question = HP.question15
start = time.time()
result = vectordb.similarity_search(question)
end = time.time()
times = (end - start)/60
log.LogChat(5, "Vector_with_sentence_simularity", question, result[0].page_content, times)
print(result[0].page_content)
print(times)

This unnamed Irish man met Rubeus Hagrid in the Leaky Cauldron in 1990. He sold Hagrid the three-headed dog he owned. Hagrid named this dog Fluffy, and, the following year, lent him to Albus Dumbledore, who used him to guard the Philosopher's Stone.[1]
0.0015552481015523275


In [63]:
question = HP.question16
start = time.time()
result = vectordb.similarity_search(question)
end = time.time()
times = (end - start)/60
log.LogChat(5, "Vector_with_sentence_simularity", question, result[0].page_content, times)
print(result[0].page_content)
print(times)

Hippogriff | Harry Potter Wiki | Fandom

















































Harry Potter Wiki





 Explore

 




 Main Page




 Discuss




All Pages




Community




Interactive Maps




Recent Blog Posts








Books

 




The Seven Novels
 




Harry Potter and the Philosopher's Stone




Harry Potter and the Chamber of Secrets




Harry Potter and the Prisoner of Azkaban




Harry Potter and the Goblet of Fire




Harry Potter and the Order of the Phoenix
0.0013714035352071126


In [64]:
question = HP.question17
start = time.time()
result = vectordb.similarity_search(question)
end = time.time()
times = (end - start)/60
log.LogChat(5, "Vector_with_sentence_simularity", question, result[0].page_content, times)
print(result[0].page_content)
print(times)

She owned a fine golden cup which was fairly innocuous aside from being her possession. The cup was sought by Lord Voldemort nearly a thousand years later, for him to turn it into a Horcrux. She was the best friend of Rowena Ravenclaw. Her house corresponds to the element of earth.[4]
0.0017510096232096354


In [65]:
question = HP.question18
start = time.time()
result = vectordb.similarity_search(question)
end = time.time()
times = (end - start)/60
log.LogChat(5, "Vector_with_sentence_simularity", question, result[0].page_content, times)
print(result[0].page_content)
print(times)

Scabbers: Ron inherited Scabbers from Percy, who was gifted of an owl (Hermes) by his father in 1991 for becoming a Prefect.[11] It was an old, grey rat which missed a toe. Scabbers was already very old when Ron got it, and even if Ron complained about it all the time, saying that Scabbers was useless, he got angry when it seemed that Crookshanks ate the rat. At the end of Ron's third year, it turned out that Scabbers was actually the unregistered Animagus Peter Pettigrew, who was believed to
0.0019565741221110027


In [66]:
question = HP.question19
start = time.time()
result = vectordb.similarity_search(question)
end = time.time()
times = (end - start)/60
log.LogChat(5, "Vector_with_sentence_simularity", question, result[0].page_content, times)
print(result[0].page_content)
print(times)

The Weasley family
0.0019035696983337401


In [67]:
question = HP.question20
start = time.time()
result = vectordb.similarity_search(question)
end = time.time()
times = (end - start)/60
log.LogChat(5, "Vector_with_sentence_simularity", question, result[0].page_content, times)
print(result[0].page_content)
print(times)

The Weasley family
0.0022242387135823567
